In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6778,2021-01-11T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1298,208,1506,41313,...,2765,NaN,NaN,109642,1311962,868928.0,NaN,18.0,NaN,NaN
6779,2021-01-11T17:00:00,ITA,9,Toscana,43.769231,11.255889,789,144,933,8087,...,3884,NaN,NaN,125330,1971143,1093335.0,NaN,10.0,Positivi diagnosticati solo con test antigenic...,NaN
6780,2021-01-11T17:00:00,ITA,10,Umbria,43.106758,12.388247,272,51,323,3949,...,664,NaN,NaN,30955,529153,260622.0,NaN,3.0,NaN,NaN
6781,2021-01-11T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,52,1,53,389,...,391,NaN,NaN,7532,65162,39852.0,NaN,0.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6778,2021-01-11T17:00:00,19,Sicilia,1298,208,1506,41313,42819,1313,1587,64058,2765,NaN,NaN,109642,1311962,868928.0,18.0,NaN,NaN
6779,2021-01-11T17:00:00,9,Toscana,789,144,933,8087,9020,-81,379,112426,3884,NaN,NaN,125330,1971143,1093335.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN
6780,2021-01-11T17:00:00,10,Umbria,272,51,323,3949,4272,-32,49,26019,664,NaN,NaN,30955,529153,260622.0,3.0,NaN,NaN
6781,2021-01-11T17:00:00,2,Valle d'Aosta,52,1,53,389,442,12,27,6699,391,NaN,NaN,7532,65162,39852.0,0.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-01-11', '2021-01-10')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-11T17:00:00,1,Piemonte,2705,177,2882,12103,14985,-357,609,184391,8195,NaN,NaN,207571,1744499,1011196.0,6.0,NaN,NaN
1,2021-01-11T17:00:00,2,Valle d'Aosta,52,1,53,389,442,12,27,6699,391,NaN,NaN,7532,65162,39852.0,0.0,NaN,NaN
2,2021-01-11T17:00:00,3,Lombardia,3522,462,3984,50523,54507,-3291,1488,421890,25849,NaN,NaN,502246,5061374,2679880.0,23.0,NaN,NaN
3,2021-01-11T17:00:00,5,Veneto,2582,358,2940,84600,87540,-1273,1715,192734,7427,NaN,NaN,287701,3477099,1262079.0,15.0,NaN,NaN
4,2021-01-11T17:00:00,6,Friuli Venezia Giulia,688,67,755,12164,12919,34,361,41747,1928,NaN,NaN,56594,989014,356255.0,8.0,NaN,52 casi già positivi al tampone antigenico son...
5,2021-01-11T17:00:00,7,Liguria,737,64,801,4281,5082,-144,226,55900,3018,NaN,NaN,64000,744811,333981.0,2.0,I 184422 test antigenici rapidi sono riferiti ...,NaN
6,2021-01-11T17:00:00,8,Emilia-Romagna,2741,238,2979,58487,61466,1558,1942,122963,8403,NaN,NaN,192832,2704917,1272633.0,14.0,NaN,NaN
7,2021-01-11T17:00:00,9,Toscana,789,144,933,8087,9020,-81,379,112426,3884,NaN,NaN,125330,1971143,1093335.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2021-01-11T17:00:00,10,Umbria,272,51,323,3949,4272,-32,49,26019,664,NaN,NaN,30955,529153,260622.0,3.0,NaN,NaN
9,2021-01-11T17:00:00,11,Marche,548,81,629,13940,14569,-677,333,30868,1711,NaN,NaN,47148,578570,346156.0,8.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-10T17:00:00,1,Piemonte,2703,176,2879,12463,15342,-487,803,183444,8176,NaN,NaN,206962,1739805,1009307.0,10.0,NaN,NaN
1,2021-01-10T17:00:00,2,Valle d'Aosta,53,1,54,376,430,-7,19,6686,389,NaN,NaN,7505,64906,39760.0,0.0,NaN,NaN
2,2021-01-10T17:00:00,3,Lombardia,3598,459,4057,53741,57798,2361,3267,417173,25787,NaN,NaN,500758,5048018,2674656.0,23.0,NaN,NaN
3,2021-01-10T17:00:00,5,Veneto,2585,355,2940,85873,88813,-638,2167,189784,7389,NaN,NaN,285986,3466098,1259295.0,37.0,NaN,NaN
4,2021-01-10T17:00:00,6,Friuli Venezia Giulia,703,64,767,12118,12885,-5,642,41450,1898,NaN,NaN,56233,986523,355337.0,7.0,NaN,110 casi già positivi al tampone antigenico so...
5,2021-01-10T17:00:00,7,Liguria,709,64,773,4453,5226,82,374,55538,3010,NaN,NaN,63774,742726,333083.0,5.0,I 182909 test antigenici rapidi sono riferiti ...,NaN
6,2021-01-10T17:00:00,8,Emilia-Romagna,2705,237,2942,56966,59908,1540,2193,122647,8337,NaN,NaN,190892,2695285,1269123.0,13.0,NaN,NaN
7,2021-01-10T17:00:00,9,Toscana,769,140,909,8192,9101,-4,472,111995,3855,NaN,NaN,124951,1965493,1091275.0,12.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2021-01-10T17:00:00,10,Umbria,265,50,315,3989,4304,67,257,25942,660,NaN,NaN,30906,528657,260437.0,7.0,NaN,NaN
9,2021-01-10T17:00:00,11,Marche,541,73,614,14632,15246,640,687,29871,1698,NaN,NaN,46815,576643,345005.0,8.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-11T17:00:00,1,Piemonte,2705,177,2882,12103,14985,-357,609,184391,8195,NaN,NaN,207571,1744499,1011196.0,6.0,NaN,NaN
1,2021-01-11T17:00:00,2,Valle d'Aosta,52,1,53,389,442,12,27,6699,391,NaN,NaN,7532,65162,39852.0,0.0,NaN,NaN
2,2021-01-11T17:00:00,3,Lombardia,3522,462,3984,50523,54507,-3291,1488,421890,25849,NaN,NaN,502246,5061374,2679880.0,23.0,NaN,NaN
3,2021-01-11T17:00:00,4,Trentino-Alto Adige,583,69,652,13052,13704,46,194,39851,1798,0.0,0.0,55353,850066,322583.0,4.0,0,NaN
4,2021-01-11T17:00:00,5,Veneto,2582,358,2940,84600,87540,-1273,1715,192734,7427,NaN,NaN,287701,3477099,1262079.0,15.0,NaN,NaN
5,2021-01-11T17:00:00,6,Friuli Venezia Giulia,688,67,755,12164,12919,34,361,41747,1928,NaN,NaN,56594,989014,356255.0,8.0,NaN,52 casi già positivi al tampone antigenico son...
6,2021-01-11T17:00:00,7,Liguria,737,64,801,4281,5082,-144,226,55900,3018,NaN,NaN,64000,744811,333981.0,2.0,I 184422 test antigenici rapidi sono riferiti ...,NaN
7,2021-01-11T17:00:00,8,Emilia-Romagna,2741,238,2979,58487,61466,1558,1942,122963,8403,NaN,NaN,192832,2704917,1272633.0,14.0,NaN,NaN
8,2021-01-11T17:00:00,9,Toscana,789,144,933,8087,9020,-81,379,112426,3884,NaN,NaN,125330,1971143,1093335.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2021-01-11T17:00:00,10,Umbria,272,51,323,3949,4272,-32,49,26019,664,NaN,NaN,30955,529153,260622.0,3.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2021-01-10T17:00:00,1,Piemonte,2703,176,2879,12463,15342,-487,...,183444,8176,NaN,NaN,206962,1739805,1009307.0,10.0,NaN,NaN
1,1,2021-01-10T17:00:00,2,Valle d'Aosta,53,1,54,376,430,-7,...,6686,389,NaN,NaN,7505,64906,39760.0,0.0,NaN,NaN
2,2,2021-01-10T17:00:00,3,Lombardia,3598,459,4057,53741,57798,2361,...,417173,25787,NaN,NaN,500758,5048018,2674656.0,23.0,NaN,NaN
3,19,2021-01-10T17:00:00,4,Trentino-Alto Adige,564,70,634,13024,13658,297,...,39710,1791,0.0,0.0,55159,848958,322073.0,1.0,0,NaN
4,3,2021-01-10T17:00:00,5,Veneto,2585,355,2940,85873,88813,-638,...,189784,7389,NaN,NaN,285986,3466098,1259295.0,37.0,NaN,NaN
5,4,2021-01-10T17:00:00,6,Friuli Venezia Giulia,703,64,767,12118,12885,-5,...,41450,1898,NaN,NaN,56233,986523,355337.0,7.0,NaN,110 casi già positivi al tampone antigenico so...
6,5,2021-01-10T17:00:00,7,Liguria,709,64,773,4453,5226,82,...,55538,3010,NaN,NaN,63774,742726,333083.0,5.0,I 182909 test antigenici rapidi sono riferiti ...,NaN
7,6,2021-01-10T17:00:00,8,Emilia-Romagna,2705,237,2942,56966,59908,1540,...,122647,8337,NaN,NaN,190892,2695285,1269123.0,13.0,NaN,NaN
8,7,2021-01-10T17:00:00,9,Toscana,769,140,909,8192,9101,-4,...,111995,3855,NaN,NaN,124951,1965493,1091275.0,12.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2021-01-10T17:00:00,10,Umbria,265,50,315,3989,4304,67,...,25942,660,NaN,NaN,30906,528657,260437.0,7.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2021-01-11T17:00:00,1,Piemonte,2705,177,2882,12103,14985,-357,609,...,8195,NaN,NaN,207571,1744499,1011196.0,6.0,NaN,NaN,4341375
1,2021-01-11T17:00:00,2,Valle d'Aosta,52,1,53,389,442,12,27,...,391,NaN,NaN,7532,65162,39852.0,0.0,NaN,NaN,125501
2,2021-01-11T17:00:00,3,Lombardia,3522,462,3984,50523,54507,-3291,1488,...,25849,NaN,NaN,502246,5061374,2679880.0,23.0,NaN,NaN,10103969
3,2021-01-11T17:00:00,4,Trentino-Alto Adige,583,69,652,13052,13704,46,194,...,1798,0.0,0.0,55353,850066,322583.0,4.0,0,NaN,1074819
4,2021-01-11T17:00:00,5,Veneto,2582,358,2940,84600,87540,-1273,1715,...,7427,NaN,NaN,287701,3477099,1262079.0,15.0,NaN,NaN,4907704
5,2021-01-11T17:00:00,6,Friuli Venezia Giulia,688,67,755,12164,12919,34,361,...,1928,NaN,NaN,56594,989014,356255.0,8.0,NaN,52 casi già positivi al tampone antigenico son...,1211357
6,2021-01-11T17:00:00,7,Liguria,737,64,801,4281,5082,-144,226,...,3018,NaN,NaN,64000,744811,333981.0,2.0,I 184422 test antigenici rapidi sono riferiti ...,NaN,1543127
7,2021-01-11T17:00:00,8,Emilia-Romagna,2741,238,2979,58487,61466,1558,1942,...,8403,NaN,NaN,192832,2704917,1272633.0,14.0,NaN,NaN,4467118
8,2021-01-11T17:00:00,9,Toscana,789,144,933,8087,9020,-81,379,...,3884,NaN,NaN,125330,1971143,1093335.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2021-01-11T17:00:00,10,Umbria,272,51,323,3949,4272,-32,49,...,664,NaN,NaN,30955,529153,260622.0,3.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2021-01-11T17:00:00,1,Piemonte,2705,177,2882,12103,14985,-357,609,...,NaN,NaN,207571,1744499,1011196.0,6.0,NaN,NaN,4341375,575
1,2021-01-11T17:00:00,2,Valle d'Aosta,52,1,53,389,442,12,27,...,NaN,NaN,7532,65162,39852.0,0.0,NaN,NaN,125501,20
2,2021-01-11T17:00:00,3,Lombardia,3522,462,3984,50523,54507,-3291,1488,...,NaN,NaN,502246,5061374,2679880.0,23.0,NaN,NaN,10103969,1036
3,2021-01-11T17:00:00,4,Trentino-Alto Adige,583,69,652,13052,13704,46,194,...,0.0,0.0,55353,850066,322583.0,4.0,0,NaN,1074819,106
4,2021-01-11T17:00:00,5,Veneto,2582,358,2940,84600,87540,-1273,1715,...,NaN,NaN,287701,3477099,1262079.0,15.0,NaN,NaN,4907704,1016
5,2021-01-11T17:00:00,6,Friuli Venezia Giulia,688,67,755,12164,12919,34,361,...,NaN,NaN,56594,989014,356255.0,8.0,NaN,52 casi già positivi al tampone antigenico son...,1211357,180
6,2021-01-11T17:00:00,7,Liguria,737,64,801,4281,5082,-144,226,...,NaN,NaN,64000,744811,333981.0,2.0,I 184422 test antigenici rapidi sono riferiti ...,NaN,1543127,209
7,2021-01-11T17:00:00,8,Emilia-Romagna,2741,238,2979,58487,61466,1558,1942,...,NaN,NaN,192832,2704917,1272633.0,14.0,NaN,NaN,4467118,634
8,2021-01-11T17:00:00,9,Toscana,789,144,933,8087,9020,-81,379,...,NaN,NaN,125330,1971143,1093335.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2021-01-11T17:00:00,10,Umbria,272,51,323,3949,4272,-32,49,...,NaN,NaN,30955,529153,260622.0,3.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-01-11T17:00:00,8,Emilia-Romagna,2741,238,2979,58487,61466,1558,1942,...,37.54,1521,-251,9632,3510.0,66,316,20.16,0.04347,4.32
1,2021-01-11T17:00:00,5,Veneto,2582,358,2940,84600,87540,-1273,1715,...,35.24,-1273,-452,11001,2784.0,38,2950,15.59,0.03495,5.86
2,2021-01-11T17:00:00,19,Sicilia,1298,208,1506,41313,42819,1313,1587,...,35.37,1280,-146,8698,5165.0,37,237,18.25,0.03194,2.21
3,2021-01-11T17:00:00,3,Lombardia,3522,462,3984,50523,54507,-3291,1488,...,44.59,-3218,-1779,13356,5224.0,62,4717,11.14,0.01473,4.97
4,2021-01-11T17:00:00,12,Lazio,2887,321,3208,75673,78881,96,1254,...,37.90,61,-492,10501,4829.0,45,1113,11.94,0.02138,3.08
5,2021-01-11T17:00:00,15,Campania,1400,109,1509,73208,74717,-671,1021,...,21.58,-697,-232,9690,6352.0,37,1655,10.54,0.01765,3.47
6,2021-01-11T17:00:00,16,Puglia,1388,169,1557,53376,54933,-346,622,...,45.80,-363,-540,3577,2770.0,26,942,17.39,0.01552,2.55
7,2021-01-11T17:00:00,1,Piemonte,2705,177,2882,12103,14985,-357,609,...,30.78,-360,-194,4694,1889.0,19,947,12.97,0.01403,4.78
8,2021-01-11T17:00:00,9,Toscana,789,144,933,8087,9020,-81,379,...,27.53,-105,-93,5650,2060.0,29,431,6.71,0.01018,3.37
9,2021-01-11T17:00:00,6,Friuli Venezia Giulia,688,67,755,12164,12919,34,361,...,37.22,46,-281,2491,918.0,30,297,14.49,0.02980,4.67


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,69,37,106,65.09
1,Marche,81,62,143,56.64
2,Puglia,169,200,369,45.80
3,Lombardia,462,574,1036,44.59
4,Umbria,51,66,117,43.59
5,Lazio,321,526,847,37.90
6,Emilia-Romagna,238,396,634,37.54
7,Friuli Venezia Giulia,67,113,180,37.22
8,Sicilia,208,380,588,35.37
9,Veneto,358,658,1016,35.24


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,2021-01-09T17:00:00,22,P.A. Trento,333,48,381,1457,1838,-92,142,20451,1006,NaN,NaN,23295,467966,154496.0,3.0,NaN,"Dei 142 casi odierni, 48 sono molecolari con p..."
642,2021-01-10T17:00:00,21,P.A. Bolzano,221,24,245,11415,11660,137,262,19081,778,NaN,NaN,31519,378840,167033.0,0.0,NaN,NaN
643,2021-01-10T17:00:00,22,P.A. Trento,343,46,389,1609,1998,160,345,20629,1013,NaN,NaN,23640,470118,155040.0,1.0,NaN,"Dei 345 casi odierni, 255 sono molecolari con ..."
644,2021-01-11T17:00:00,21,P.A. Bolzano,233,24,257,11453,11710,50,125,19155,779,NaN,NaN,31644,379596,167387.0,0.0,NaN,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,2021-01-07T17:00:00,4,Trentino-Alto Adige,566,72,638,12559,13197,68,433,...,6.0,4361.0,960.0,64.0,11.29,4.83,9.93,45.10,0.00,0.52
319,2021-01-08T17:00:00,4,Trentino-Alto Adige,563,70,633,12648,13281,84,243,...,10.0,2716.0,576.0,89.0,11.06,4.77,8.95,42.19,-2.78,0.64
320,2021-01-09T17:00:00,4,Trentino-Alto Adige,541,72,613,12748,13361,80,522,...,17.0,5874.0,1183.0,100.0,11.75,4.59,8.89,44.13,2.86,0.60
321,2021-01-10T17:00:00,4,Trentino-Alto Adige,564,70,634,13024,13658,297,607,...,7.0,4319.0,1016.0,276.0,11.04,4.64,14.05,59.74,-2.78,2.22


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
